# Non-negative Matrix Factorization

In this notebook, there are a number of attempts at topic modeling on coffee reviews. This included making different NMF models based on different subsets of text, different numbers of topics, and using a CountVectorizer or TFIDF embedding

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import re
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict 

In [6]:
#data wrangling packages
from sklearn.manifold import TSNE
import random 
random.seed(13)

#visualization packages
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline

import nltk
# nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
with open('coffee_words.pickle','rb') as read_file:
    coffee = pickle.load(read_file)
with open('coffee_ratings.pickle','rb') as read_file:
    ratings = pickle.load(read_file)
ratings = ratings.reset_index().rename(columns={'index':'Roaster'})
with open('df_full.pickle','rb') as read_file:
    df = pickle.load(read_file)
with open('df_topic_breakdown.pickle','rb') as read_file:
    df_topic_breakdown = pickle.load(read_file)
    
with open('blindtfidf_vec.pickle', 'rb') as read_file:
    blindtfidf = pickle.load(read_file)
with open('blindtfidf_mat.pickle', 'rb') as read_file:
    tfidf_blind = pickle.load(read_file)

In [8]:
from nltk.corpus import stopwords
sw = stopwords.words("english")
sw = sw + ['coffee','coffees','cup','john', 'diruocco','jen','apodaca','ken','kevin','keurig','espresso','serve','capsule','device','serving','flavor','notes','mouthfeel','aroma','finish','brewed','brewing','parts','one','two','three','evaluate','evaluated','hint']

In the cell above, the stopwords from the NLTK corpus (and, the, it, etc.) are augmented with some specific to the reviews found in this corpus that are either ubiqutous (coffee, cup), unrelated to the coffee (john, jen, etc.), relate to a category and not the coffee itself (flavor, notes, etc.), or are related to the preparation and not the analysis (keurig, espresso, etc). 

### Corpus by Coffees

Below, you can see the first few reviews in the dataframe. While numerous attempts were made concerning all the text, just reviews, just notes, or just summaries, the eventual conclusion is that the Review portion of the website, which is a blind assessment, most succintly and clearly describes the experience of drinking the coffee.

In [6]:
coffee.head()

,Roaster,Review,Notes,TLDR
0,Jackrabbit Java,"Yeasty, richly sweet-savory. Fresh-baked bread...",Processed by the anaerobic natural method (who...,An anaerobically processed coffee with rich st...
1,Jackrabbit Java,"Balanced, sweet-toned, floral. Tea rose, cocoa...",Produced at Mahembe Farm and processed at the ...,"A friendly, classic Rwanda cup: balanced, deep..."
2,Red Rooster Coffee Roaster,"Delicate, deep; complex. Pomegranate, macadami...",This coffee earned the highest rating in a cup...,An intricately original natural-processed Colo...
3,Paradise Roasters,"Very sweet, floral-toned. Freesia, pink grapef...",This coffee tied for the second-highest rating...,A sweetly evocative presentation of Colombia P...
4,Kakalove Cafe,"Opulent, richly sweet-tart-savory. Black curra...",This coffee tied for the second-highest rating...,"A rich, complex, decadently sweet cup processe..."


In [10]:
# Function used to display topics from NMF models
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

## Let's start seeing about potential topics for the reviews!

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
nmf_tfidffull.components_

In [ ]:
nmf_tfidffull= NMF(6)
fulltfidf_topic = nmf_tfidffull.fit_transform(tfidf_full)
fulltopic_tfidf = pd.DataFrame(nmf_tfidffull.components_.round(3),
#              index = ["component_1","component_2"],
             columns = fulltfidf.get_feature_names())
display_topics(nmf_tfidffull, fulltfidf.get_feature_names(), 10)

## NMF on blind reviews

In [ ]:
nmf_blind= NMF(3)
blinddoc_topic = nmf_blind.fit_transform(blind_word)
blindtopic_word = pd.DataFrame(nmf_blind.components_.round(3),
#              index = ["component_1","component_2"],
             columns = blindvectorizer.get_feature_names())
display_topics(nmf_blind, blindvectorizer.get_feature_names(), 10)

In [ ]:
nmf_tfidfblind= NMF(9)
blindtfidf_topic = nmf_tfidfblind.fit_transform(tfidf_blind)
blindtopic_tfidf = pd.DataFrame(nmf_tfidfblind.components_.round(3),
#              index = ["component_1","component_2"],
             columns = blindtfidf.get_feature_names())
display_topics(nmf_tfidfblind, blindtfidf.get_feature_names(), 10)

In [ ]:
blindtfidf_topic

In [ ]:
with open('nmf_tfidfblind.pickle', 'wb') as to_write:
    pickle.dump(nmf_tfidfblind, to_write)

with open('blindtfidf_topic.pickle', 'wb') as to_write:
    pickle.dump(blindtfidf_topic, to_write)

with open('blindtopic_tfidf.pickle', 'wb') as to_write:
    pickle.dump(blindtopic_tfidf, to_write)

## NMF on TLDR
min_df=20, max_df=1000 with 7 or 8 topics starts getting what seem to be real profiles
Reducing max_df from 3000 to 1000 only eliminates 4 words from corpus (coffee, cup, fruit, sweet)
3000 to 1500 only removes cup, to 1200 removes coffee and cup

In [ ]:
nmf_tldr2 = NMF(2)
tldrdoc_topic2 = nmf_tldr2.fit_transform(tldr_word)
tldrtopic_word2 = pd.DataFrame(nmf_tldr2.components_.round(3),
             index = ["component_1","component_2"],
             columns = tldrvectorizer.get_feature_names())
display_topics(nmf_tldr2, tldrvectorizer.get_feature_names(), 10)

## NMF on the full text of all reviews, notes, and summaries

### Notes for two topics
We begin to see more common terms of robust notes chocolate, rich, dark, roast in the first topic
We see more delicate terms in the second like fruit, floral, ethiopia, washed

In [ ]:
nmf_model2 = NMF(2)
doc_topic2 = nmf_model2.fit_transform(doc_word)

In [ ]:
topic_word = pd.DataFrame(nmf_model2.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word.sort_values(by='component_1',axis=1,ascending=False)

In [ ]:
display_topics(nmf_model2, vectorizer.get_feature_names(), 10)

In [ ]:
nmf_model3 = NMF(3)
doc_topic3 = nmf_model3.fit_transform(doc_word)
topic_word = pd.DataFrame(nmf_model3.components_.round(3),
             index = ["component_1","component_2","component_3"],
             columns = vectorizer.get_feature_names())
display_topics(nmf_model3, vectorizer.get_feature_names(), 10)

## Assignments to the different topics for each review were added to the original text datarame for further analysis

In [ ]:
blindtfidf_topic[0]

In [ ]:
topic_vect = pd.DataFrame(blindtfidf_topic,columns=['bright_floral_citrus','choc_woody_dark','tart_sweet_smooth','cacao_nut_clean','sweet_nut_pine','juicy_cacao_honey','red_berries','woody_nut_caramel','cherry_vinuous_choc'])

In [ ]:
df_topic_breakdown = pd.DataFrame()
df_topic_breakdown['roaster'] = df['Roaster']
df_topic_breakdown['origin'] = ratings['Coffee Origin']
df_topic_breakdown['roast_level'] = ratings['Roast Level']
df_topic_breakdown['rating'] = ratings.Overall
df_topic_breakdown['length'] = coffee.Review.str.replace(r'\d+','',regex=True).str.len()
df_topic_breakdown['word count'] = blind_word.todense().sum(axis=1)
df_topic_breakdown['group'] = df['Topic']
df_topic_breakdown = pd.concat([df_topic_breakdown,topic_vect],axis=1)

In [ ]:
with open('df_topic_breakdown.pickle', 'wb') as to_write:
    pickle.dump(df_topic_breakdown, to_write)

In [ ]:
counts = df['Topic'].value_counts()

counts.plot(kind='bar')

In [ ]:
df[df['Ten Topic'] == 1].iloc[15].Text

## Below is a plot of the distribution of assignments to the various topics from two topics down to ten topics

This was completed on some initial versions of the NMF topics as a potential use for predicting coffee scores, but did not ultimately carry as much weight there or in gaining interpretation in topics. The plot below was valuable to better understand how coffee reviews eventually were distributed across a variety of topics, moreso to understand NMF than the coffees.

In [ ]:
two = df['Two Topic'].value_counts()
three = df['Three Topic'].value_counts()
four = df['Four Topic'].value_counts()
five = df['Five Topic'].value_counts()
six = df['Six Topic'].value_counts()
seven = df['Seven Topic'].value_counts()
eight = df['Eight Topic'].value_counts()
nine = df['Nine Topic'].value_counts()
ten = df['Ten Topic'].value_counts()

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15,10))
fig.suptitle('Distributions of Topics from Text')

two.plot(ax=axes[0,0], kind='bar')
three.plot(ax=axes[0,1], kind='bar')
four.plot(ax=axes[0,2], kind='bar')
five.plot(ax=axes[1,0], kind='bar')
six.plot(ax=axes[1,1], kind='bar')
seven.plot(ax=axes[1,2], kind='bar')
eight.plot(ax=axes[2,0], kind='bar')
nine.plot(ax=axes[2,1], kind='bar')
ten.plot(ax=axes[2,2], kind='bar')

## Using the three topic model above as an example, ratings from the ratings data set can be compiled for comparison across the different topics.

In [ ]:
combined = pd.DataFrame()
combined['roaster'] = df['Roaster']
combined['origin'] = ratings['Coffee Origin']
combined['roast_level'] = ratings['Roast Level']
combined['group'] = df['Topic']
combined['rating'] = ratings.Overall
combined['aroma'] = ratings.Aroma
combined['body'] = ratings.Body
combined['flavor'] = ratings.Flavor
combined['aftertaste'] = ratings.Aftertaste
combined['acidity'] = ratings.Acidity
combined.sample(5)

In [ ]:
# How does the overall rating of coffee compare across the three topics?
combined.loc[combined.rating == 'NR','rating'] = '-999'
combined.rating = combined.rating.astype(int)
combined[combined.rating>0].groupby(by='group').rating.mean()

In [ ]:
# How does the aroma rating of coffee compare across the three topics?
combined.loc[(combined.aroma == 'NR')|(combined.aroma == 'NA'),'aroma'] = '-999'
combined.aroma = combined.aroma.astype(float)
combined.aroma = combined.aroma.round(0)
combined[combined.aroma>0].groupby(by='group').aroma.mean()

In [ ]:
# How does the body rating of coffee compare across the three topics?
combined.loc[(combined.body == 'NR')|(combined.body == 'NA'),'body'] = '-999'
combined.body = combined.body.astype(float)
combined.body = combined.body.round(0)
combined[combined.body>0].groupby(by='group').body.mean()

In [ ]:
# How does the flavor rating of coffee compare across the three topics?
combined.loc[(combined.flavor == 'NR')|(combined.flavor == 'NA'),'flavor'] = '-999'
combined.flavor = combined.flavor.astype(float)
combined.flavor = combined.flavor.round(0)
combined[combined.flavor>0].groupby(by='group').flavor.mean()

In [ ]:
# How does the aftertaste rating of coffee compare across the three topics?
combined.aftertaste.fillna('-999',inplace=True)
combined.aftertaste = combined.aftertaste.astype(float)
combined.aftertaste = combined.aftertaste.round(0)
combined[combined.aftertaste>0].groupby(by='group').aftertaste.mean()

In [ ]:
# How does the acidity rating of coffee compare across the three topics?
combined.loc[(combined.acidity == 'NR')|(combined.acidity == 'NA')|(combined.acidity == 'na')|(combined.acidity == 'n/a'),'acidity'] = '-999'
combined.loc[(combined.acidity == 'Very Low'),'acidity'] = '1'
combined.loc[(combined.acidity == 'Low'),'acidity'] = '3'
combined.loc[(combined.acidity == 'Moderate'),'acidity'] = '5'
combined.acidity.fillna(-999,inplace=True)
combined.acidity = combined.acidity.astype(float)
combined.acidity = combined.acidity.round(0)
combined[combined.acidity>0].groupby(by='group').acidity.mean()

In [ ]:
topic_df = combined[combined['origin'].isnull() == False][['roaster','origin','roast_level','group']]
topic_df.sample(20)

In [ ]:
with open('combined.pickle', 'wb') as to_write:
    pickle.dump(combined, to_write)

In [ ]:
combined.head()

In [ ]:
df['rating'] =  combined['rating'] 

In [ ]:
with open('df.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

In [ ]:
feature_matrix = vectorizer.transform(df.Text.str.replace(r'\d+','',regex=True))
#Transform the data
data = nmf_model10.transform(feature_matrix.todense())
data

In [ ]:
feature_matrix = vectorizer.transform(df.Text.str.replace(r'\d+','',regex=True))
#Transform the data
data = nmf_model2.transform(feature_matrix.todense())
data

label = doc_topic2.argmax(axis=1)
#Getting unique labels
#plotting the results:
plt.figure(figsize=(10,8))
for i in range(0,10):
    plt.scatter(data[label == i , 0] , data[label == i , 1] , label = i)
plt.legend(['Light Roast','Dark Roast'],fontsize='large')
plt.show()

In [ ]:
label = blindtfidf_topic.argmax(axis=1)
#Getting unique labels
#plotting the results:
plt.figure(figsize=(12,8))
for i in range(0,10):
    plt.scatter(data[label == i , 0] , data[label == i , 1] , label = i)
plt.legend(['Ethiopia','Dark','Fair Trade','Easy Brew','Processing','Tart/Cocoa','Peaberries','Rich/Floral','Geisha/Floral','Kenya/Fruit'],fontsize='large')
plt.show()

In [ ]:
feature_matrix = blindtfidf.transform(coffee.Review.str.replace(r'\d+','',regex=True))
#Transform the data
data = nmf_tfidfblind.transform(feature_matrix.todense())

label = blindtfidf_topic.argmax(axis=1)
#Getting unique labels
#plotting the results:
plt.figure(figsize=(12,8))
for i in range(0,10):
    plt.scatter(data[label == i , 0] , data[label == i , 1] , label = i)
plt.legend(['Natural Ethiopia (Floral/Fruit)','Vinuous/Complex','Sweet/Savory/Tart','Espresso Drink','Darker Roast/Choc','Floral/Spice/Nut','Kenya/Bright/Acidic/Juicy','Citrus/Floral/Woody'],fontsize='large')
plt.show()

## Coffees with strongest links to the topics

In [ ]:
nmf_embedding = nmf_tfidfblind.transform(tfidf_blind)
nmf_embedding = (nmf_embedding - nmf_embedding.mean(axis=0))/nmf_embedding.std(axis=0)

top_idx = np.argsort(nmf_embedding,axis=0)[-3:]

count = 0
for idxs in top_idx.T: 
    print("\nTopic {}:".format(count))
    for idx in idxs:
        print(ratings.iloc[idx]['Roaster'],": ",ratings.iloc[idx]['Coffee Origin'])
    count += 1

## T-SNE Visualization

In [ ]:
topics = ['Bright, Floral, Citrus', 'Chocolate, Dark, Woody', 'Tart, Sweet, Smooth','Cacao, Nutty, Clean', 'Sweet, Hazelnut, Pine', 'Juicy, Honey, Cacao', 'Red Berries','Nutty, Caramel, Woody', 'Cherry, Vinous, Chocolate']

In [ ]:
tsne10 = TSNE(random_state=3211,metric='cosine',perplexity=10,n_iter=5000,n_iter_without_progress=250)
tsne10_embedding = tsne10.fit_transform(nmf_embedding)
tsne10_embedding = pd.DataFrame(tsne10_embedding,columns=['x','y'])
tsne10_embedding['hue'] = nmf_embedding.argmax(axis=1)

# tsne30 = TSNE(random_state=3211,metric='cosine',perplexity=30,n_iter=5000,n_iter_without_progress=250)
# tsne30_embedding = tsne30.fit_transform(nmf_embedding)
# tsne30_embedding = pd.DataFrame(tsne30_embedding,columns=['x','y'])
# tsne30_embedding['hue'] = nmf_embedding.argmax(axis=1)

# tsne50 = TSNE(random_state=3211,metric='cosine',perplexity=50,n_iter=5000,n_iter_without_progress=250)
# tsne50_embedding = tsne50.fit_transform(nmf_embedding)
# tsne50_embedding = pd.DataFrame(tsne30_embedding,columns=['x','y'])
# tsne50_embedding['hue'] = nmf_embedding.argmax(axis=1)

In [ ]:
from matplotlib import cm
colors = np.array([list(cm.tab10(0)),
                  list(cm.tab10(1)),
                  list(cm.tab10(2)),
                  list(cm.tab10(3)),
                  list(cm.tab10(4)),
                  list(cm.tab10(5)),
                  list(cm.tab10(6)),
                  list(cm.tab10(7)),
                  list(cm.tab10(8)),
                  ])


legend_list = []

for i in range(len(topics)):   
    color = colors[i]
    legend_list.append(mpatches.Ellipse((0, 0), 1, 1, fc=color))

In [ ]:
# Perplexity of 10
matplotlib.rc('font',family='monospace')

fig, axs = plt.subplots(1,1, figsize=(15, 15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .1, wspace=0)

count = 0
legend = []

data = tsne10_embedding
scatter = axs.scatter(data=data,x='x',y='y',s=6,c=data['hue'],cmap='tab10')
axs.set_title('Blind Reviews from Coffee-Review.com',**{'fontsize':'10'})
axs.axis('off')

plt.suptitle("All Coffee Reviews Clustered by Category",**{'fontsize':'14','weight':'bold'})
plt.figtext(.51,0.95,'Unsupervised topic modeling with NMF based on textual content + 2D-embedding with t-SNE:', **{'fontsize':'10','weight':'light'}, ha='center')


fig.legend(legend_list,topics,loc=(0.2,0.89),ncol=3)
plt.subplots_adjust(top=0.85)
plt.savefig(r'C:\Users\ejfel\Documents\metis_repos\Coffee-Reviews-NLP\tsne_nmf10')
plt.show()

In [ ]:
# Perplexity of 30
matplotlib.rc('font',family='monospace')

fig, axs = plt.subplots(1,1, figsize=(15, 15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .1, wspace=0)

count = 0
legend = []

data = tsne30_embedding
scatter = axs.scatter(data=data,x='x',y='y',s=6,c=data['hue'],cmap='tab10')
axs.set_title('Blind Reviews from Coffee-Review.com',**{'fontsize':'10'})
axs.axis('off')

plt.suptitle("All Coffee Reviews Clustered by Category",**{'fontsize':'14','weight':'bold'})
plt.figtext(.51,0.95,'Unsupervised topic modeling with NMF based on textual content + 2D-embedding with t-SNE:', **{'fontsize':'10','weight':'light'}, ha='center')


fig.legend(legend_list,topics,loc=(0.2,0.89),ncol=3)
plt.subplots_adjust(top=0.85)
plt.savefig(r'C:\Users\ejfel\Documents\metis_repos\Coffee-Reviews-NLP\tsne_nmf30')
plt.show()

In [ ]:
# Perplexity of 50
matplotlib.rc('font',family='monospace')

fig, axs = plt.subplots(1,1, figsize=(15, 15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .1, wspace=0)

count = 0
legend = []

data = tsne50_embedding
scatter = axs.scatter(data=data,x='x',y='y',s=6,c=data['hue'],cmap='tab10')
axs.set_title('Blind Reviews from Coffee-Review.com',**{'fontsize':'10'})
axs.axis('off')

plt.suptitle("All Coffee Reviews Clustered by Category",**{'fontsize':'14','weight':'bold'})
plt.figtext(.51,0.95,'Unsupervised topic modeling with NMF based on textual content + 2D-embedding with t-SNE:', **{'fontsize':'10','weight':'light'}, ha='center')


fig.legend(legend_list,topics,loc=(0.2,0.89),ncol=3)
plt.subplots_adjust(top=0.85)
plt.savefig(r'C:\Users\ejfel\Documents\metis_repos\Coffee-Reviews-NLP\tsne_nmf50')
plt.show()

In [ ]:
# tsne = TSNE(random_state=3211,metric='cosine')
# tsne_embedding = tsne.fit_transform(nmf_embedding)
# tsne_embedding = pd.DataFrame(tsne_embedding,columns=['x','y'])
# tsne_embedding['hue'] = nmf_embedding.argmax(axis=1)
from itertools import cycle
X = nmf_embedding

model = TSNE(n_components=2, random_state=0,verbose=0, metric='cosine')
low_data = model.fit_transform(X)

target = df_topic_breakdown.group


colors = cycle(['r','g','b','c','m','y','orange','k','aqua','yellow'])
target_ids = range(len(topics))
plt.figure(dpi=150)
for i, c, label in zip(target_ids, colors, topics):
    plt.scatter(low_data[target == i, 0], low_data[target == i, 1], c=c, label=label, s=15, alpha=1)
plt.legend(fontsize=8, bbox_to_anchor=(1.05, 1.0), loc='upper left', frameon=True, facecolor='#FFFFFF', edgecolor='#333333')
plt.xlim(-100,100);
plt.title("Coffee Description Clusters with TSNE", fontsize=12)
plt.ylabel("Junk TSNE Axis 2", fontsize=12)
plt.xlabel("Junk TSNE Axis 1", fontsize=12);
plt.xticks(fontsize=10)
plt.yticks(fontsize=10);